In [98]:
import pandas as pd
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup as bs
import requests
import time
from sqlalchemy import *
from sqlalchemy import create_engine
import sqlite3 as sql

import os
os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs')

## To restart, continue from year 2006 and Q4

In [99]:
def latest_sec_db_links(formtype, year, qtr, engine):
    
    if formtype == 'F4':
        table_name = 'F4Links'
    elif formtype == '10Q':
        table_name = 'Links10Q'
    elif formtype == '10K':
        table_name = 'Links10K'
    else:
        print('formtypes are F4 10Q and 10K')
        return
    
    query = 'SELECT * FROM {0} WHERE Year = {1} AND Quarter = {2}'.format(table_name, year, qtr)
    
    db_table = pd.read_sql_query(query, con=engine, index_col = 'idx')
    
    db_table['DateFiled'] = pd.to_datetime(db_table['DateFiled'])
    
    return db_table.reset_index(drop = True)

sec_engine = create_engine('sqlite:///SEC.db', echo=False)



In [100]:
year = 2003
qtr = 3

for year in range(2003, 2019):
    for qtr in range(1,5):
        start_time = time.time()
        if year == 2003 and qtr < 3:
            continue
        else:
            latest_db_10q = latest_sec_db_links('10Q', year, qtr, sec_engine)
            
            htms = pd.DataFrame(columns = latest_db_10q.columns.tolist() + ['Filename'])
            txts = pd.DataFrame(columns = latest_db_10q.columns.tolist() + ['Filename'])
            nones = pd.DataFrame(columns = latest_db_10q.columns.tolist() + ['Filename'])
            
            for idx in latest_db_10q.index:
                crawler_url = latest_db_10q.loc[idx,'URL']
                crawler_docs = bs(requests.get(crawler_url).text,'lxml').select_one('table[class="tableFile"]')
                crawler_10q_url = 'https://www.sec.gov' + crawler_docs.find_all('tr')[1].find('a').attrs['href']
                filetype = crawler_10q_url.split('.')[-1]
                if filetype == 'htm':
                    htms.loc[idx,:] = latest_db_10q.loc[idx,:]
                    htms.loc[idx,'Filename'] = crawler_10q_url
                elif filetype == 'txt':
                    txts.loc[idx,:] = latest_db_10q.loc[idx,:]
                    txts.loc[idx,'Filename'] = crawler_10q_url
                else:
                    nones.loc[idx,:] = latest_db_10q.loc[idx,:]
                    nones.loc[idx,'Filename'] = crawler_10q_url

            htms = htms.reset_index(drop = True)
            txts = txts.reset_index(drop = True)
            nones = nones.reset_index(drop = True)

            sec_htm_engine = create_engine('sqlite:///SEC_htm.db', echo=False)
            
            htms.to_sql('Links10Qs', con=sec_htm_engine, if_exists='append', index_label = 'idx')
            txts.to_sql('Links10Qs_txts', con=sec_htm_engine, if_exists='append', index_label = 'idx')
            nones.to_sql('Links10Qs_nas', con=sec_htm_engine, if_exists='append', index_label = 'idx')
        print('Completed in {} seconds'.format(time.time() - start_time))
        print('For {0} Q{1}'.format(year,qtr))

Completed in 1952.8416481018066 seconds
For 2003 Q3
Completed in 1882.1644442081451 seconds
For 2003 Q4
Completed in 467.19344758987427 seconds
For 2004 Q1
Completed in 1930.4655978679657 seconds
For 2004 Q2
Completed in 1793.1308660507202 seconds
For 2004 Q3
Completed in 1936.799148797989 seconds
For 2004 Q4
Completed in 428.4852285385132 seconds
For 2005 Q1
Completed in 1812.6493921279907 seconds
For 2005 Q2
Completed in 1819.385198354721 seconds
For 2005 Q3
Completed in 1812.4467849731445 seconds
For 2005 Q4
Completed in 442.94344305992126 seconds
For 2006 Q1
Completed in 1752.1853849887848 seconds
For 2006 Q2
Completed in 1723.0692150592804 seconds
For 2006 Q3


KeyboardInterrupt: 